# Generation Using Idyoms

In [1]:
using DataFrames, Plots, Random
using Chakra
include("src/Idyoms.jl")
using Main.Idyoms

## Load Melch

In [2]:
using Melch
Melch.__INIT__("/Users/nick/Dropbox/MelchData")

## Select Nova Dataset

In [3]:
nova = fnd(Melch.id(0),Melch)
nova.description

"Songs and ballads from Nova Scotia, Canada."

## Generate Sequences

In [4]:
melodies = pts(nova)
sequences = [sequence(pts(mel,Melch),Melch) for mel in melodies];

In [5]:
dump(sequences[1][1])

Melch.Event
  id: Melch.EventId
    dataset: Int64 0
    melody: Int64 1
    event: Int64 1
  ONSET: Int64 0
  DELTAST: Int64 0
  BIOI: Int64 1
  DUR: Int64 24
  CPITCH: Int64 65
  MPITCH: Int64 38
  ACCIDENTAL: Int64 0
  KEYSIG: Int64 -1
  MODE: Int64 0
  BARLENGTH: Int64 48
  PULSES: Int64 2
  PHRASE: Int64 1
  VOICE: Int64 1
  ORNAMENT: Int64 0
  COMMA: Int64 0
  VERTINT12: Chakra.None Chakra.None()
  ARTICULATION: Int64 0
  DYN: Chakra.None Chakra.None()


## Define Viewpoints

In [6]:
CPITCH = vp(Melch.CPITCH)
DUR = vp(Melch.DUR)
CPITCH_X_DUR = link(CPITCH,DUR)

Chakra.LinkedViewpoint{Tuple{Int64, Int64}}(Viewpoint[Chakra.AtomicViewpoint{Int64}(Melch.Attribute{:CPITCH, Int64}(), Type[Int64]), Chakra.AtomicViewpoint{Int64}(Melch.Attribute{:DUR, Int64}(), Type[Int64])], DataType[Int64, Int64])

## Generate Views

In [7]:
p_p = [View(seq,CPITCH,CPITCH) for seq in sequences];
d_d = [View(seq,DUR,DUR) for seq in sequences];
p_d = [View(seq,CPITCH,DUR) for seq in sequences];
d_p = [View(seq,DUR,CPITCH) for seq in sequences];
pxd_pxd = [View(seq,CPITCH_X_DUR,CPITCH_X_DUR) for seq in sequences];

## Set Parameters

In [8]:
o = Bounded(5) # order bound
b = Interpolated() # smoothing technique
e = A() # escape method
u = true # update exclusion

true

## Generate Models

In [9]:
p_p_tally, p_seen = Idyoms.train(p_p,o);
d_d_tally, d_seen = Idyoms.train(d_d,o);
p_d_tally, d_seen = Idyoms.train(p_d,o);
d_p_tally, p_seen = Idyoms.train(d_p,o);
pxd_pxd_tally, pxd_seen = Idyoms.train(pxd_pxd,o);

## Use models to generate new sequences

In [10]:
preds, seq = Idyoms.gen_seq(100,pxd_pxd_tally,pxd_seen,pxd_seen,b,e,u,o);

LoadError: UndefVarError: generate_sequence not defined

## Display Sequence

In [11]:
Idyoms.mean_infcontent(preds)

LoadError: UndefVarError: preds not defined

In [12]:
df = Idyoms.todataframe(1,preds)

LoadError: UndefVarError: preds not defined

In [13]:
plot(df.Prob)
plot!(df.IC)
plot!(df.H)
plot!(size=(900,300))

LoadError: UndefVarError: df not defined

In [14]:
using MIDI
function generatemidifile(xs::Vector{Tuple{Int,Int}},name::String)
    tpq = 960
    notes = Note[]
    pos = 0
    for pair in xs
        pitch = pair[1]
        velocity = 100
        dur = Int(tpq*pair[2]/24)
        
        note = Note(pitch,100,pos,dur)
        
        push!(notes,note)
        pos+=dur
    end
    track = MIDITrack()
    addnotes!(track, notes)
    
    midi = MIDIFile(0, tpq, [track])
    cd(@__DIR__)
    save(name, midi)
end

generatemidifile (generic function with 1 method)

In [15]:
generatemidifile(seq,"example.mid")

LoadError: UndefVarError: seq not defined